In [6]:
import ipython_microblaze as ipmb
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [7]:
%%microblaze_functions base.ARDUINO
#include <pyprintf.h>
#include <arduino.h>
#include "xgpio_l.h"
#include "xgpio.h"

// select IO locations for button scan
#define IO0 (1<<0)
#define IO1 (1<<1)
#define IO2 (1<<2)
#define IO3 (1<<3)
#define IO4 (1<<4)
#define IO5 (1<<5)
#define IO6 (1<<6)
#define IO7 (1<<7)
#define IO8 (1<<8)
#define IO9 (1<<9)
#define IO10 (1<<10)

// gpio variable
XGpio gpo;

// 2d matrix button scan decode
// the second number is the row, first is a column
static u8 buttonLookup[4][4] = {
    {15,11,7,3},
    {14,10,6,2},
    {13,9,5,1},
    {12,8,4,0}
};

void setup_keypad(void) {
        
    //initialize arduino MB    
    arduino_init(0,1,0,0);

    //config switch    
    config_arduino_switch(A_GPIO, A_GPIO, A_GPIO, A_GPIO, A_GPIO, A_GPIO,
                               D_GPIO, D_GPIO, D_GPIO, D_GPIO, D_GPIO,
                               D_GPIO, D_GPIO, D_GPIO, D_GPIO,
                               D_SS, D_MOSI, D_MISO, D_SPICLK);

    // setup GPIO port for button scan, C columns read, R rows write
    XGpio_Initialize(&gpo, XPAR_GPIO_0_DEVICE_ID);
    XGpio_SetDataDirection(&gpo, 1, 0xFF0F);  // columns are lower in the byte
}

    //these are the current key colors
    static u8 i=0,j=0,k=0,l=0,n=0,o=0,p=0,q=0,r=0,s=0,t=0,u=0,v=0,w=0,x=0,y=0;

//note to self
//byte rowPins[ROWS] = {6,7,8,9}; //connect to the row pinouts of the keypad
//byte colPins[COLS] = {3,4,5,11}; //connect to the column pinouts of the keypad

static void _spi_transfer(u32 BaseAddress, int bytecount,
                  u8* readBuffer, u8* writeBuffer) {
    int i;
    XSpi_WriteReg(BaseAddress,XSP_SSR_OFFSET, 0xfe);
    for (i=0; i<bytecount; i++){
        //pyprintf("In Write: %d\n", i);
        if (writeBuffer) {
            XSpi_WriteReg(BaseAddress,XSP_DTR_OFFSET, writeBuffer[i]);
        } else {
            XSpi_WriteReg(BaseAddress,XSP_DTR_OFFSET, 0);
        }
    }
    //while(((XSpi_ReadReg(BaseAddress,XSP_SR_OFFSET) & 0x04)) != 0x04);
        //for(volatile int i = 0; i < 10; ++i);// Add this
         while(((XSpi_ReadReg(BaseAddress,XSP_SR_OFFSET) & 0x05)) != 0x04);
    
    // Read SPI
    for(i=0;i< bytecount; i++){
        if (readBuffer) {
           readBuffer[i] = XSpi_ReadReg(BaseAddress,XSP_DRR_OFFSET);
        } else {
            XSpi_ReadReg(BaseAddress,XSP_DRR_OFFSET);
        }
    }

    XSpi_WriteReg(BaseAddress, XSP_SSR_OFFSET, 0xff);
}

// send the color word to the keypad
int keycolor16(int keynum, int colornum )
{
    u8 tempo[8]; 
    u8 scratch[8]; 

    int go=0;
    //this is the secret decoder ring for the key colors
    u8 colormap[11]={0b0000,0b0001,0b0011,0b0111,0b0110,0b1000,0b1010,0b1110,0b1001,0b1011,0b1111};
    //off, red, org, yel, grn, dkblu, lt blu, aqua, pink, lt pink, wht

 //pick the color variable to modify basedon the key number   
    switch (keynum){          
        case 0:
            i=colornum;
            go=1;
        break;
        case 1:
            j=colornum;
            go=1;
        break;
        case 2:
            k=colornum;
            go=1;
        break;
        case 3:
            l=colornum;
            go=1;
        break;
        case 4:
            n=colornum;
            go=1;
        break;
        case 5:
            o=colornum;
            go=1;
        break;
        case 6:
            p=colornum;
            go=1;
        break;
        case 7:
            q=colornum;
            go=1;
        break;
        case 8:
            r=colornum;
            go=1;
        break;
        case 9:
            s=colornum;
            go=1;
        break;
        case 10:
            t=colornum;
            go=1;
        break;
        case 11:
            u=colornum;
            go=1;
        break;
        case 12:
            v=colornum;
            go=1;
        break;
        case 13:
            w=colornum;
            go=1;
        break;
        case 14:
            x=colornum;
            go=1;
        break;
        case 15:
            y=colornum;
            go=1;
        break;
        default:
            go=0;
            
    }
    //if the set was valid (go=1)
    if(go){
        // build color control word
        tempo[0]=(0b00001111&colormap[i]) +((0b00001111&colormap[j])<<4);  
        tempo[1]=(0b00001111&colormap[k]) +((0b00001111&colormap[l])<<4);              
        tempo[2]=(0b00001111&colormap[n]) +((0b00001111&colormap[o])<<4);           
        tempo[3]=(0b00001111&colormap[p]) +((0b00001111&colormap[q])<<4);           
        tempo[4]=(0b00001111&colormap[r]) +((0b00001111&colormap[s])<<4);            
        tempo[5]=(0b00001111&colormap[t]) +((0b00001111&colormap[u])<<4);       
        tempo[6]=(0b00001111&colormap[v]) +((0b00001111&colormap[w])<<4);      
        tempo[7]=(0b00001111&colormap[x]) +((0b00001111&colormap[y])<<4);    
    //send word
        for (int ii=0;ii<8;ii++){
            _spi_transfer(DIRECT_SPI_BASEADDR, 8, scratch, tempo);
        }          
    }
    
   return(0);
}

// increment when pressing a button
// I think this initializes everything to zero
u8 buttonstate[16] = {0};

// scan the keypad
unsigned int getkey(void){
    u8 c, keynum=0;
    int i = IO7;

    for(u8 r=4; r>0;){
        r--;
        XGpio_DiscreteWrite(&gpo, 1, (~i));
        int read = XGpio_DiscreteRead(&gpo,1);
        c = 0;

        if ((read&IO0)==0) {
            c = 1;
        } else if ((read&IO1)==0) {
            c = 2;
        } else if ((read&IO2)==0) {
            c = 3;
        } else if ((read&IO3)==0) {
            c = 4;
        }
        
        // detected a valid button press, output the number
        if (c!=0) {
            c--;
            keynum = buttonLookup[c][r];
            keynum++;
            break;
        }
        i=i>>1;
    }

    return(keynum);
}

In [8]:
   %%microblaze_functions base.PMODA
#include <pmod.h>
#include "xgpio.h"
#include <pyprintf.h>
#include <timer.h>
// select IO locations for button scan
#define MISOp (1<<0)
#define SCKp (1<<1)
#define MOSIp (1<<2)
#define SSp (1<<3)

#define PMOD_DIO_BASEADDR 0x40000000
XGpio gpo;

void setup_bitbang(void){
    
    
    //MISO,  SPICLK,MOSI,  SS
    config_pmod_switch(GPIO_0,GPIO_1,GPIO_2,GPIO_3,
                       GPIO_4,GPIO_5,GPIO_6,GPIO_7);
    pmod_init(0,0);
    
    XGpio_Initialize(&gpo,PMOD_DIO_BASEADDR);
 //setup port   
    XGpio_SetDataDirection(&gpo, 1,0x00000001);
    XGpio_DiscreteWrite(&gpo, 1, 0);    //set evryone low
  //  XGpio_DiscreteWrite(&gpo, 1, SSp);    //set SS high
 //bitbang
    

}


int bitbangSPI(int cmd){
   int count=0,tempo=0;
   u8 dbit=0; 
    
    
    XGpio_DiscreteWrite(&gpo, 1, 0b00000001 ) ;   //set SS low
    for(volatile int i = 0; i < 200; ++i);// delay
    
    for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );
  
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );
  
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        tempo=XGpio_DiscreteRead(&gpo,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low
        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));
        

        
    } //second byte
             for(volatile int i = 0; i < 400; ++i);// delay
        count=0;
        for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );
  
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );
  
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        tempo=XGpio_DiscreteRead(&gpo,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low
        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));
        
    } 
    
        int tempo_out=count;
    
    //third byte
             for(volatile int i = 0; i < 400; ++i);// delay
        count=0;
        for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );
  
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        tempo=XGpio_DiscreteRead(&gpo,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo, 1, 0b00000001 | dbit<< 2 );    //set clk low

        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));
        
    } 
    

    for(volatile int i = 0; i < 200; ++i);// delay    
    
    XGpio_DiscreteWrite(&gpo, 1, SSp);    //set SS high  
    
    return tempo_out;
    
}  

In [9]:
   %%microblaze_functions base.PMODB
#include <pmod.h>
#include "xgpio.h"
#include <pyprintf.h>
#include <timer.h>
// select IO locations for button scan
#define MISOp (1<<0)
#define SCKp (1<<1)
#define MOSIp (1<<2)
#define SSp (1<<3)

#define PMOD_DIO_BASEADDR 0x40000000
XGpio gpo_disp;

void setup_bb_display(void){
    
    //MISO,  SPICLK,MOSI,  SS
    config_pmod_switch(GPIO_0,GPIO_1,GPIO_2,GPIO_3,
                       GPIO_4,GPIO_5,GPIO_6,GPIO_7);
    pmod_init(0,0);
    
    XGpio_Initialize(&gpo_disp,PMOD_DIO_BASEADDR);
 //setup port   
    XGpio_SetDataDirection(&gpo_disp, 1,0x00000001);
    XGpio_DiscreteWrite(&gpo_disp, 1, 0);    //set evryone low
  //  XGpio_DiscreteWrite(&gpo_disp, 1, SSp);    //set SS high
 //bitbang
    
}

// create the bitbang spi command from page and instrument inputs
int bb_spi_display(int d_instrument, int d_page){
    int count=0,tempo=0;
    u8 dbit=0;
    u8 cmd = (((d_instrument&0xF)<<4)+(d_page&0xF));

    XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 ) ;   //set SS low
    for(volatile int i = 0; i < 200; ++i);// delay

    for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
        tempo=XGpio_DiscreteRead(&gpo_disp,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));
        
    } //second byte
    count=0;
    for(volatile int i = 0; i < 400; ++i);// delay
    for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
            
        tempo=XGpio_DiscreteRead(&gpo_disp,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));
        
    } 
    
        int tempo_out=count;
    
    //third byte
    count=0;
    for(volatile int i = 0; i < 400; ++i);// delay
    for (int i=0;i<8;i++){
        // set data bit 
        if (cmd & 1<<(7-i))
             dbit=1;    //set databit 
        else
             dbit=0;    //set databit low
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
        tempo=XGpio_DiscreteRead(&gpo_disp,1)&0b00000001; //rising edge read here
        
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000011 | dbit<< 2 );    //set clk high
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        XGpio_DiscreteWrite(&gpo_disp, 1, 0b00000001 | dbit<< 2 );    //set clk low
        
        if (tempo)
            count= (count | (1<<(7-i)));
        else
            count =(count & ~(1<<(7-i)));

    } 

    for(volatile int i = 0; i < 200; ++i);// delay    
    XGpio_DiscreteWrite(&gpo_disp, 1, SSp);    //set SS high      
    return tempo_out;
    
}  

In [10]:
setup_keypad()
setup_bitbang()
setup_bb_display()

In [11]:
import time
from pynq.lib import Button

while (base.buttons[0].read()==0):

    count1=bitbangSPI(1)
    count2=bitbangSPI(2)
    
    keycolor16(count1%16,count2%10)
    

In [ ]:
import time
from pynq.lib import Button

while (base.buttons[0].read()==0):
    count1=bitbangSPI(1)
    count2=bitbangSPI(2)
    bb_spi_display(count1%5+1,count2%4+1)
    
    

In [ ]:
import time
count1=bitbangSPI(1)
print(count1)
ii=0
jj=0
while (base.buttons[0].read()==0):
    keycolor16(ii%16,jj%10)
    ii=ii+1
    jj=jj+3
    time.sleep(.25)
    

In [ ]:
while (base.buttons[0].read()==0):
    key=getkey()
    print(key, end="   \r")

led_states = []

for i in range(16):
    led_states.append(0)
    keycolor16(i,0)

import time
from pynq.lib import Button

while (base.buttons[0].read()==0):
    time.sleep(0.1)
    keyin = 0
    while keyin is 0:
        keyin = getkey()
        time.sleep(0.1)
    keyin-=1
    led_states[keyin]+=1
    if (led_states[keyin] > 10):
        led_states[keyin] = 0
    for i in range(16):
        keycolor16(i,led_states[i])